In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/ColabNotebooks/MIMIC/MEEP

In [ ]:
pip install bayesian-optimization

In [ ]:
import random 
import torch.nn as nn 
import numpy as np 
import pandas as pd 
import scipy.stats as st
import argparse
import sys
import os
import json
import pickle 
from tqdm import tqdm
from bayes_opt import BayesianOptimization
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score
import scipy.stats as ss
import rocutils.roc_utils as rr
from rocutils.roc_utils import *

# Date
from datetime import date
today = date.today()
date = today.strftime("%m%d")

# plotting 
import matplotlib 
import matplotlib.pyplot as plt
from palettable.cmocean.sequential import Dense_20
from mpl_toolkits.axes_grid1 import make_axes_locatable
plt.style.use("bmh")
matplotlib.rcParams["figure.dpi"] = 300
plt.rcParams["font.weight"] = "bold"
plt.rcParams["axes.labelweight"] = "bold"
legend_properties = {'weight':'bold', 'size': 10}
f_sm = nn.Softmax(dim=1)
kf = KFold(n_splits=10, random_state=42, shuffle=True)

# make dir and write json files 
def write_json(target_path, target_file, data):
    if not os.path.exists(target_path):
        try:
            os.makedirs(target_path)
        except Exception as e:
            print(e)
            raise
    with open(os.path.join(target_path, target_file), 'w') as f:
        json.dump(data, f)

# count model trainable params 
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# combine train and val and based on the cross validation index, redistribute train and val 
def get_cv_data(train_data, dev_data, train_target, dev_target, train_index, dev_index):
    trainval_head = train_data + dev_data
    trainval_static = np.concatenate((train_target, dev_target), axis=0)
    train_cv = [trainval_head[i] for i in train_index]
    train_cvl = [trainval_static[i] for i in train_index]
    dev_cv = [trainval_head[i] for i in dev_index]
    dev_cvl = [trainval_static[i] for i in dev_index]
    return train_cv, dev_cv, np.asarray(train_cvl), np.asarray(dev_cvl)

# calcuate accuracy 
def cal_acc(pred, label):
    pred_t = torch.concat(pred)
    prediction =  torch.argmax(pred_t, dim=-1).unsqueeze(-1)
    label_t = torch.concat(label)
    acc = (prediction == label_t).sum()/len(pred_t)
    return acc

# calcualte accuracy for positive classes 
def cal_pos_acc(pred, label, pos_ind):
    pred_t = torch.concat(pred)
    prediction =  torch.argmax(pred_t, dim=-1).unsqueeze(-1)
    label_t = torch.concat(label)
    # positive index
    ind = [i for i in range(len(pred_t)) if label_t[i] == pos_ind]
    acc = (prediction[ind] == label_t[ind]).sum()/len(ind)
    return acc

# random sampling 
class DictDist():
    def __init__(self, dict_of_rvs): self.dict_of_rvs = dict_of_rvs
    def rvs(self, n):
        a = {k: v.rvs(n) for k, v in self.dict_of_rvs.items()}
        out = []
        for i in range(n): out.append({k: vs[i] for k, vs in a.items()})
        return out

# filter ICU stays based on LOS needed: 48+6, 4+6, 12+6
def filter_los(static_data, vitals_data, thresh, gap):
    # (200, 80)
    los = [i.shape[1] for i in vitals_data]
    ind = [i for i in range(len(los)) if los[i]>=(thresh+gap) and np.isnan(static_data[i, 0]) == False]
    vitals_reduce = [vitals_data[i][:, :thresh] for i in ind] 
    static_data = static_data[ind]
    return static_data, vitals_reduce

# filter for the ARF tasks 
def filter_arf(args, vital):
    vital_reduce = []
    target = []
    for i in range(len(vital)):
        arf_flag = np.where(vital[i][184, :] == 1)[0]
        peep_flag = np.union1d(np.where(vital[i][157, :] == 1)[0], np.where(vital[i][159, :] == 1)[0])
        if len(arf_flag) == 0:
            if len(peep_flag) > 0:
                if peep_flag[0] >= (args.thresh + args.gap):
                    vital_reduce.append(vital[i][:, :args.thresh])
                    target.append(1)
            else:
                vital_reduce.append(vital[i][:, :args.thresh])
                target.append(0)
        elif len(arf_flag) > 0:
            if arf_flag[0] >= (args.thresh + args.gap):
                if (len(peep_flag) > 0 and peep_flag[0] >= (args.thresh + args.gap)) or len(peep_flag) == 0:
                    vital_reduce.append(vital[i][:, :args.thresh])
                    target.append(1)
    return vital_reduce, np.asarray(target)

# filter for the shock task 
def filter_shock(args, vital):
    vital_reduce = []
    target = []
    for i in range(len(vital)):
        shock_flag = np.where(vital[i][186:191].sum(axis=0) >= 1)[0]
        if len(shock_flag) == 0:
            vital_reduce.append(vital[i][:, :args.thresh])
            target.append(0)
        elif len(shock_flag) > 0:
            if shock_flag[0] >= (args.thresh + args.gap):
                vital_reduce.append(vital[i][:, :args.thresh])
                target.append(1)
    return vital_reduce, np.asarray(target)

In [ ]:
# set data path for both MIMIC and eICU 
datapath = '/content/drive/MyDrive/ColabNotebooks/MIMIC/Extract/MEEP/Extracted_Feb_2023/MIMIC_compile_0218_2023_2.npy'
datapath_cv = '/content/drive/MyDrive/ColabNotebooks/MIMIC/Extract/MEEP/Extracted_Feb_2023/MIMIC_compile_0218_2023_2.npy'
# Target 0: hospital mortality; 1: ARF; 2: Shock 
# Threshold list by hour: 48, 4, 12
# Gap hour: 6 
# Models : LR, RF
target_list = [0, 1, 1, 2, 2]
thresh_list = [24, 2, 6, 2, 6]
gap_list = [4, 4, 4, 4, 4]
model_list = ['LR', 'RF']
output_classes = 2

# empty result_dict
result_dict = {}

# load data
data_label = np.load(datapath, allow_pickle=True).item()
train_head = data_label['train_head']
static_train_filter = data_label['static_train_filter']
dev_head = data_label['dev_head']
static_dev_filter = data_label['static_dev_filter']
test_head = data_label['test_head']
static_test_filter = data_label['static_test_filter']
s_train = np.stack(static_train_filter, axis=0)
s_dev = np.stack(static_dev_filter, axis=0)
s_test = np.stack(static_test_filter, axis=0)

# load cross validation data 
data_label = np.load(datapath_cv, allow_pickle=True).item()
etrain_head = data_label['train_head']
estatic_train_filter = data_label['static_train_filter']
edev_head = data_label['dev_head']
estatic_dev_filter = data_label['static_dev_filter']
etest_head = data_label['test_head']
estatic_test_filter = data_label['static_test_filter']
es_train = np.stack(estatic_train_filter, axis=0)
es_dev = np.stack(estatic_dev_filter, axis=0)
es_test = np.stack(estatic_test_filter, axis=0) 

for m in range(2): # model choices 
    args.model_name = model_list[m]
    for ii in range(0, 5): # tasks 
        result_dict = {}
        args.thresh = thresh_list[ii]
        args.target_index = target_list[ii]
        args.gap = gap_list[ii]

        print('Running target %d, thresh %d, gap %d, model %s'%(args.target_index, args.thresh, args.gap, args.model_name))
        workname = date + args.model_name + '_target_%d'%args.target_index + '_%dh'%args.thresh + '_%dh'%args.gap
        print(workname)

        if args.target_index == 0 and args.filter_los == True:
            print('Before filtering, train size is %d'%(len(train_head)))
            train_label, train_data = filter_los(s_train, train_head, args.thresh, args.gap)
            dev_label, dev_data = filter_los(s_dev, dev_head, args.thresh, args.gap)
            test_label, test_data = filter_los(s_test, test_head, args.thresh, args.gap)
            print('After filtering, train size is %d'%(len(train_data)))
            train_label = train_label[:, 0]
            dev_label = dev_label[:, 0]
            test_label = test_label[:, 0]

        if args.target_index == 1 : # arf
            print('Before filtering, train size is %d'%(len(train_head)))
            train_data, train_label = filter_arf(args, train_head)
            dev_data, dev_label= filter_arf(args, dev_head)
            test_data, test_label = filter_arf(args, test_head)
            print('After filtering, train size is %d'%(len(train_data)))

        if args.target_index == 2 : # shock
            print('Before filtering, train size is %d'%(len(train_head)))
            train_data, train_label = filter_shock(args, train_head)
            dev_data, dev_label = filter_shock(args, dev_head)
            test_data, test_label = filter_shock(args, test_head)
            print('After filtering, train size is %d'%(len(train_data)))

        trainval_target = np.concatenate((train_label, dev_label), axis=0).astype(int)
        trainval_data = np.stack(train_data + dev_data, axis=0).reshape((len(trainval_target), -1))

        # generate weight for the imbalanced label distirbution 
        if args.model_name == 'LR':
            def bo_tune_rf(C, l1_ratio):
                regressor = LogisticRegression(penalty='elasticnet', dual=False, C=C, fit_intercept=True, intercept_scaling=1, \
                                                class_weight='balanced', random_state=0, solver='saga', \
                                                verbose=0, warm_start=False, n_jobs=None, l1_ratio=l1_ratio)
            
                scores = cross_val_score(regressor, trainval_data, trainval_target, cv=5, scoring='roc_auc')
                return np.mean(scores)

            #Invoking the Bayesian Optimizer with the specified parameters to tune
            xgb_bo = BayesianOptimization(bo_tune_rf, {
                                                'C' : (0.01, 5), 
                                                'l1_ratio': (0, 1)})

            #performing Bayesian optimization for 5 iterations with 8 steps of random exploration with an #acquisition function of expected improvement
            xgb_bo.maximize(n_iter=10, init_points=5, acq='ei')
            #   0.9368   |  0.745    |  620.2    |  0.00842  C     | l1_ratio  | max_iter  |    tol  
            p_dict = xgb_bo.max['params']
            # p_dict = {'C': 0.1803, 'l1_ratio': 0.9982}
            classifier = LogisticRegression(penalty='elasticnet', dual=False, C =p_dict['C'], fit_intercept=True, intercept_scaling=1, \
                                                class_weight='balanced', random_state=0, solver='saga', \
                                                verbose=0, warm_start=False, n_jobs=None, l1_ratio=p_dict['l1_ratio'])
            classifier.fit(trainval_data, trainval_target)


        elif args.model_name == 'RF':
            
            #Baysian Optimization 
        # 'max_depth': 6, 'eta':0.3, 'objective':'reg:squarederror', 'num_parallel_tree': 100, 'subsample': 0.5, 
        #           'reg_lambda': 2, 'reg_alpha':1, 'colsample_bytree': 0.5
            def bo_tune_rf(n_estimators, max_depth, min_samples_split, min_samples_leaf, max_features, max_samples):

                regressor = RandomForestClassifier(n_estimators=int(n_estimators), max_depth=int(max_depth), \
                                                    min_samples_split= int(min_samples_split), min_samples_leaf=int(min_samples_leaf), \
                                                    max_features = max_features, \
                                                    random_state=0, verbose=0, max_samples=max_samples, class_weight='balanced')
                
                scores = cross_val_score(regressor, trainval_data, trainval_target, cv=2, scoring='roc_auc')
                # cv_result = xgb.cv(params, dtrain, num_boost_round=int(num_round), nfold=5)
                #Return the negative RMSE
                return np.mean(scores)

            #Invoking the Bayesian Optimizer with the specified parameters to tune
            xgb_bo = BayesianOptimization(bo_tune_rf, {
                                                'n_estimators' : (50, 200),
                                                'max_depth': (3, 30), 
                                                'min_samples_split': (2, 11),  
                                                'min_samples_leaf': (1, 11), 
                                                'max_features': (0.2, 1), 
                                                'max_samples': (0.2, 1)})

            #performing Bayesian optimization for 5 iterations with 8 steps of random exploration with an #acquisition function of expected improvement
            xgb_bo.maximize(n_iter=10, init_points=5, acq='ei')
        
            p_dict = xgb_bo.max['params']
        #   |  0.8618   |  13.71    |  0.2364   |  0.5648   |  9.199    |  3.71     |  176.6    |
            # p_dict = {'n_estimators': 176.6, 'max_depth': 13.71, 'min_samples_split': 3.71, 'min_samples_leaf': 9.199, 'max_features': 0.2364, 'max_samples':0.5648}

            classifier = RandomForestClassifier(n_estimators=int(p_dict['n_estimators']), max_depth=int(p_dict['max_depth']), \
                                                        min_samples_split= int(p_dict['min_samples_split']), min_samples_leaf=int(p_dict['min_samples_leaf']), \
                                                        max_features = p_dict['max_features'], \
                                                        random_state=0, verbose=0, max_samples=p_dict['max_samples'], class_weight='balanced')
            classifier.fit(trainval_data, trainval_target)

        ## bootstrapping on its own testset 
        roc = []
        prc = []
        for i in tqdm(range(1000)):
            test_index = np.random.choice(len(test_label), 1000)
            test_i = [test_data[i] for i in test_index]
            test_t = test_label[test_index].astype(int)
            test_stack = np.stack(test_i).reshape((len(test_t), -1))
            test_pred_prob= classifier.predict_proba(test_stack)
            test_roc = roc_auc_score(test_t, test_pred_prob[:, 1])
            test_prc = average_precision_score(test_t, test_pred_prob[:, 1])
            roc.append(test_roc)
            prc.append(test_prc)

        #create 95% confidence interval for population mean weight
        print('Running target %d, thresh %d'%(args.target_index, args.thresh))
        print('%.3f'%np.mean(roc))
        print('(%.3f-%.3f)'%st.t.interval(alpha=0.95, df=len(roc), loc=np.mean(roc), scale=np.std(roc)))
        print('%.3f'%np.mean(prc))
        print('(%.3f-%.3f)'%st.t.interval(alpha=0.95, df=len(prc), loc=np.mean(prc), scale=np.std(prc)))
        result_dict[workname] = ['%.3f'%np.mean(roc)]
        result_dict[workname].append('(%.3f-%.3f)'%st.t.interval(alpha=0.95, df=len(roc), loc=np.mean(roc), scale=np.std(roc)))
        result_dict[workname].append('%.3f'%np.mean(prc))
        result_dict[workname].append('(%.3f-%.3f)'%st.t.interval(alpha=0.95, df=len(prc), loc=np.mean(prc), scale=np.std(prc)))
        result_dict[workname].append(len(test_label))

        # cross validation on the other dataset 
        if args.target_index == 0 and args.filter_los == True:
            print('Before filtering, train size is %d'%(len(etrain_head)))
            es_train, etrain_data = filter_los(es_train, etrain_head, args.thresh, args.gap)
            es_dev, edev_data = filter_los(es_dev, edev_head, args.thresh, args.gap)
            es_test, etest_data = filter_los(es_test, etest_head, args.thresh, args.gap)
            print('After filtering, train size is %d'%(len(etrain_head)))
            etrain_label = es_train[:, 0]
            edev_label= es_dev[:, 0]
            etest_label = es_test[:, 0]

        elif args.target_index == 1:
            etrain_data, etrain_label = filter_arf(args, etrain_head)
            edev_data, edev_label = filter_arf(args, edev_head)
            etest_data, etest_label = filter_arf(args, etest_head)

        elif args.target_index == 2:
            etrain_data, etrain_label = filter_shock(args, etrain_head)
            edev_data, edev_label = filter_shock(args, edev_head)
            etest_data, etest_label = filter_shock(args, etest_head)

        crossval_target = np.concatenate((etrain_label, edev_label, etest_label), axis= 0)
        crossval_head = np.stack(etrain_data + edev_data + etest_data, axis=0)

        roc = []
        prc = []
        for i in tqdm(range(1000)):
            test_index = np.random.choice(len(crossval_target), 1000)
            test_i = [crossval_head[i] for i in test_index]
            test_t = crossval_target[test_index].astype(int)
            test_stack = np.stack(test_i).reshape((len(test_t), -1))
            test_pred_prob= classifier.predict_proba(test_stack)
            test_roc = roc_auc_score(test_t, test_pred_prob[:, 1])
            test_prc = average_precision_score(test_t, test_pred_prob[:, 1])
            roc.append(test_roc)
            prc.append(test_prc)
        #create 95% confidence interval for population mean weight
        print('%.3f'%np.mean(roc))
        print('(%.3f-%.3f)'%st.t.interval(alpha=0.95, df=len(roc), loc=np.mean(roc), scale=np.std(roc)))
        print('%.3f'%np.mean(prc))
        print('(%.3f-%.3f)'%st.t.interval(alpha=0.95, df=len(prc), loc=np.mean(prc), scale=np.std(prc)))
        result_dict[workname].append('%.3f'%np.mean(roc))
        result_dict[workname].append('(%.3f-%.3f)'%st.t.interval(alpha=0.95, df=len(roc), loc=np.mean(roc), scale=np.std(roc)))
        result_dict[workname].append('%.3f'%np.mean(prc))
        result_dict[workname].append('(%.3f-%.3f)'%st.t.interval(alpha=0.95, df=len(prc), loc=np.mean(prc), scale=np.std(prc)))    
        result_dict[workname].append(len(crossval_target))
        print(result_dict) 

        # figure importance 
        LR models
        if args.model_name == 'LR':
            a = classifier.coef_
            a_pos = np.abs(a)
            c = a_pos.reshape(200, int(args.thresh)).T
            d = c.sum(axis=0)
            rf_mimic_IM = pd.DataFrame({'name': index, 'im':d})
            rf_mimic_IM.sort_values(by=['im'], ascending=False, inplace=True)
            rf_mimic_IM.to_hdf(os.path.join('./checkpoints/', 'MEEP_eICU_LRRF.h5'), key=workname)
            fig, ax = plt.subplots()
            im = ax.imshow(c, cmap=Dense_20.mpl_colormap, vmax = 5e-2, label='23 h')
            divider = make_axes_locatable(ax)
            cax = divider.append_axes('right', size='5%', pad=0.05)
            fig.colorbar(im, cax=cax, orientation='vertical')
            im.set_label('Label via method')
            # ax.legend([], ['23h'], loc='upper left', bbox_to_anchor=(0, 1.1), prop=legend_properties)
            # ax.set_title('RF model', size=12,  fontweight='bold')
            ax.grid('off')
            ax.set_ylabel('ICU_in Hours', size=12,  fontweight='bold')
            ax.set_xlabel('Feature Index', size=12,  fontweight='bold')
            plt.savefig('./checkpoints/' + workname + '_feature_importance_eicu.eps',format='eps', bbox_inches = 'tight', pad_inches = 0.1)
            plt.show()

        else: 
            a_pos = classifier.feature_importances_
            c = a_pos.reshape(200, int(args.thresh)).T
            d = c.sum(axis=0)
            rf_mimic_IM = pd.DataFrame({'name': index, 'im':d}) 
            rf_mimic_IM.to_hdf(os.path.join('./checkpoints/', 'MEEP_eICU_LRRF.h5'), key=workname)
            fig, ax = plt.subplots()
            im = ax.imshow(c, cmap=Dense_20.mpl_colormap, vmax = 5e-3, label='23 h')
            divider = make_axes_locatable(ax)
            cax = divider.append_axes('right', size='5%', pad=0.05)
            fig.colorbar(im, cax=cax, orientation='vertical')
            im.set_label('Label via method')
            # ax.legend([], ['23h'], loc='upper left', bbox_to_anchor=(0, 1.1), prop=legend_properties)
            # ax.set_title('RF model', size=12,  fontweight='bold')
            ax.grid('off')
            ax.set_ylabel('ICU_in Hours', size=12,  fontweight='bold')
            ax.set_xlabel('Feature Index', size=12,  fontweight='bold')
            plt.savefig('./checkpoints/' + workname + '_feature_importance_eicu.eps',format='eps', bbox_inches = 'tight', pad_inches = 0.1)
            plt.show()
        
        ## auc curves 
        test_stack = np.stack(test_data).reshape((len(test_label), -1))
        test_pred_prob= classifier.predict_proba(test_stack)
        test_stack_e = np.stack(crossval_head).reshape((len(crossval_target), -1))
        test_pred_prob_e= classifier.predict_proba(test_stack_e)

        ratio_1 = 1000/len(test_label)
        ratio_2 = 1000/len(crossval_target)
        rocs = rr._roc.compute_roc_bootstrap(X=test_pred_prob[:, 1], y=test_label, frac = ratio_1, pos_label=1,
                                    n_bootstrap=1000,
                                random_state=42,
                                    return_mean=False)
        rocs_1 = rr._roc.compute_roc_bootstrap(X=test_pred_prob_e[:, 1], y=crossval_target, frac= ratio_2, pos_label=1,
                                    n_bootstrap=1000,
                                    random_state=42,
                                    return_mean=False) 
        plot_mean_roc_2(rocs, rocs_1, legend='%s, %s'%(args.model_name, task_map[args.target_index]), workname = workname + '_eicu', show_ci=True, show_ti=True)


        write_json('./checkpoints', workname + '_mimic.json', result_dict)

Running target 0, thresh 24, gap 4, model RF
0228RF_target_0_24h_4h
Before filtering, train size is 27136
After filtering, train size is 12768
|   iter    |  target   | max_depth | max_fe... | max_sa... | min_sa... | min_sa... | n_esti... |
-------------------------------------------------------------------------------------------------


<ipython-input-4-5d726b46a94a>:93: DeprecationWarning: 
Passing acquisition function parameters or gaussian process parameters to maximize
is no longer supported, and will cause an error in future releases. Instead,
please use the "set_gp_params" method to set the gp params, and pass an instance
 of bayes_opt.util.UtilityFunction using the acquisition_function argument

  xgb_bo.maximize(n_iter=10, init_points=5, acq='ei')


| 1         | 0.8586    | 18.32     | 0.909     | 0.4668    | 10.93     | 6.948     | 60.84     |
| 2         | 0.8481    | 8.833     | 0.8579    | 0.3113    | 1.302     | 9.166     | 68.48     |
| 3         | 0.86      | 10.41     | 0.6027    | 0.8276    | 6.539     | 8.28      | 88.17     |
| 4         | 0.8253    | 3.306     | 0.6078    | 0.2407    | 10.76     | 3.839     | 75.89     |
| 5         | 0.8559    | 20.41     | 0.8732    | 0.3598    | 3.043     | 3.645     | 122.4     |
| 6         | 0.865     | 19.65     | 0.4684    | 0.9004    | 10.47     | 6.391     | 60.56     |
| 7         | 0.8631    | 24.18     | 0.2       | 1.0       | 8.904     | 4.571     | 59.62     |
| 8         | 0.8603    | 18.85     | 0.7133    | 0.6355    | 7.32      | 2.439     | 56.26     |
| 9         | 0.8611    | 23.05     | 0.2       | 1.0       | 10.57     | 10.09     | 55.28     |
| 10        | 0.8579    | 22.19     | 0.2       | 1.0       | 4.077     | 9.574     | 61.57     |
| 11        | 0.865 

100%|██████████| 1000/1000 [01:24<00:00, 11.87it/s]


Running target 0, thresh 24
0.870
(0.838-0.902)
0.498
(0.399-0.597)
{'0228RF_target_0_24h_4h': ['0.870', '(0.838-0.902)', '0.498', '(0.399-0.597)', 3691]}
Running target 1, thresh 2, gap 4, model RF
0228RF_target_1_2h_4h
Before filtering, train size is 27136
After filtering, train size is 6544
|   iter    |  target   | max_depth | max_fe... | max_sa... | min_sa... | min_sa... | n_esti... |
-------------------------------------------------------------------------------------------------


...

Running target 2, thresh 2
0.882
(0.834-0.930)
0.492
(0.364-0.621)
{'0228RF_target_2_2h_4h': ['0.882', '(0.834-0.930)', '0.492', '(0.364-0.621)', 5405]}
Running target 2, thresh 6, gap 4, model RF
0228RF_target_2_6h_4h
Before filtering, train size is 27136
After filtering, train size is 18518
|   iter    |  target   | max_depth | max_fe... | max_sa... | min_sa... | min_sa... | n_esti... |
-------------------------------------------------------------------------------------------------


<ipython-input-4-5d726b46a94a>:93: DeprecationWarning: 
Passing acquisition function parameters or gaussian process parameters to maximize
is no longer supported, and will cause an error in future releases. Instead,
please use the "set_gp_params" method to set the gp params, and pass an instance
 of bayes_opt.util.UtilityFunction using the acquisition_function argument

  xgb_bo.maximize(n_iter=10, init_points=5, acq='ei')


| 1         | 0.8842    | 15.52     | 0.9059    | 0.8258    | 6.067     | 6.355     | 71.14     |
| 2         | 0.8804    | 18.67     | 0.3846    | 0.2415    | 3.146     | 7.265     | 114.0     |
| 3         | 0.8909    | 26.23     | 0.4325    | 0.6636    | 6.098     | 5.621     | 118.0     |
| 4         | 0.8874    | 14.79     | 0.9832    | 0.635     | 6.584     | 3.954     | 63.53     |
| 5         | 0.8789    | 17.68     | 0.3369    | 0.7721    | 1.889     | 7.029     | 76.51     |
| 6         | 0.8864    | 25.39     | 0.5729    | 0.789     | 4.943     | 5.894     | 117.9     |
| 7         | 0.8944    | 26.83     | 0.5221    | 0.9781    | 7.11      | 5.573     | 118.3     |
| 8         | 0.8968    | 29.3      | 0.2839    | 0.9257    | 9.419     | 4.493     | 118.0     |
| 9         | 0.8957    | 28.34     | 0.446     | 0.3279    | 10.05     | 7.185     | 122.8     |
| 10        | 0.8996    | 28.64     | 0.2249    | 0.5189    | 10.35     | 10.98     | 117.0     |
| 11        | 0.8989

100%|██████████| 1000/1000 [01:00<00:00, 16.49it/s]

Running target 2, thresh 6
0.898
(0.842-0.954)
0.492
(0.329-0.656)
{'0228RF_target_2_6h_4h': ['0.898', '(0.842-0.954)', '0.492', '(0.329-0.656)', 5297]}


In [ ]:
task_map = {0: 'hosp_mort', 1: 'ARF', 2: 'shock'}
import os
import pandas as pd
total_stats = pd.read_hdf(os.path.join('/content/drive/My Drive/ColabNotebooks/MIMIC/Extract', 'MEEP_stats_0702.h5'), 
                          key = 'total_filling_after_removal')
total_stats_1 = pd.read_hdf(os.path.join('/content/drive/My Drive/ColabNotebooks/MIMIC/Extract', 'MEEP_stats_0702.h5'), 
                       key = 'mimic_intvention_filling')
index = np.concatenate((total_stats.index,total_stats_1['col_name']))
_DEFAULT_OBJECTIVE = "minoptsym"
def plot_mean_roc_2(rocs, rocs_1, legend, workname = 'MIMIC_CNN_hosp_mort', auto_flip=True, show_all=False, ax=None, **kwargs):
    """
    Compute and plot the mean ROC curve for a sequence of ROC containers.
    rocs:       List of ROC containers created by compute_roc().
    auto_flip:  See compute_roc(), applies only to mean ROC curve.
    show_all:   If True, show the single ROC curves.
                If an integer, show the rocs[:show_all] roc curves.
    show_ci:    Show confidence interval
    show_ti:    Show tolerance interval
    kwargs:     Forwarded to plot_roc(), applies only to mean ROC curve.
    Optional kwargs argument show_opt can be either False, True or a string
    specifying the particular objective function to be used to plot the
    optimal point. See get_objective() for details. Default choice is the
    "minopt" objective.
    """
    import matplotlib.pyplot as plt
    if ax is None:
        ax = plt.gca()

    n_samples = len(rocs)

    # Some default values.
    zorder = kwargs.get("zorder", 1)
    label = kwargs.pop("label", "eICU test set")
    label_1 = kwargs.pop("label", "MIMIC Whole")
    # kwargs for plot_roc()...
    show_details = kwargs.get("show_details", False)
    show_opt = kwargs.pop("show_opt", False)
    show_ti = kwargs.pop("show_ti", True)
    show_ci = kwargs.pop("show_ci", True)
    color = kwargs.pop("color", "darkorange")
    color_1 = kwargs.pop("color", "royalblue")
    is_opt_str = isinstance(show_opt, (str, list, tuple))
    # Defaults for mean-ROC.
    resolution = kwargs.pop("resolution", 101)
    objective = show_opt if is_opt_str else _DEFAULT_OBJECTIVE

    # Compute average ROC.
    ret_mean = compute_mean_roc(rocs=rocs,
                                resolution=resolution,
                                auto_flip=auto_flip,
                                objective=objective)
    ret_mean_1 = compute_mean_roc(rocs=rocs_1,
                                resolution=resolution,
                                auto_flip=auto_flip,
                                objective=objective)

    # Plot ROC curve for single bootstrap samples.
    if show_all:
        def isint(x):
            return isinstance(x, int) and not isinstance(x, bool)
        n_loops = show_all if isint(show_all) else np.inf
        n_loops = min(n_loops, len(rocs))
        for ret in rocs[:n_loops]:
            ax.plot(ret.fpr, ret.tpr,
                    color="gray",
                    alpha=0.2,
                    zorder=zorder + 2)
        for ret in rocs_1[:n_loops]:
            ax.plot(ret.fpr, ret.tpr,
                    color="gray",
                    alpha=0.2,
                    zorder=zorder + 2)
    if show_ti:
        # 95% interval
        tpr_sort = np.sort(ret_mean.tpr_all, axis=0)
        tpr_lower = tpr_sort[int(0.025 * n_samples), :]
        tpr_upper = tpr_sort[int(0.975 * n_samples), :]
        label_int = "95% of all samples" if show_details else None
        ax.fill_between(ret_mean.fpr, tpr_lower, tpr_upper,
                        color="gray", alpha=.2,
                        label=label_int,
                        zorder=zorder + 1)
        tpr_sort_1 = np.sort(ret_mean_1.tpr_all, axis=0)
        tpr_lower_1 = tpr_sort_1[int(0.025 * n_samples), :]
        tpr_upper_1= tpr_sort_1[int(0.975 * n_samples), :]
        label_int = "95% of all samples" if show_details else None
        ax.fill_between(ret_mean_1.fpr, tpr_lower_1, tpr_upper_1,
                        color="gray", alpha=.2,
                        label=label_int,
                        zorder=zorder + 1)
    if show_ci:
        # 95% confidence interval
        tpr_std = np.std(ret_mean.tpr_all, axis=0, ddof=1)
        tpr_lower = ret_mean.tpr - 1.96 * tpr_std / np.sqrt(n_samples)
        tpr_upper = ret_mean.tpr + 1.96 * tpr_std / np.sqrt(n_samples)
        label_ci = "95% CI of mean curve" if show_details else None
        ax.fill_between(ret_mean.fpr, tpr_lower, tpr_upper,
                        color=color, alpha=.3,
                        label=label_ci,
                        zorder=zorder)
        tpr_std_1 = np.std(ret_mean_1.tpr_all, axis=0, ddof=1)
        tpr_lower_1 = ret_mean_1.tpr - 1.96 * tpr_std_1 / np.sqrt(n_samples)
        tpr_upper_1 = ret_mean_1.tpr + 1.96 * tpr_std_1 / np.sqrt(n_samples)
        label_ci = "95% CI of mean curve" if show_details else None
        ax.fill_between(ret_mean_1.fpr, tpr_lower_1, tpr_upper_1,
                        color=color_1, alpha=.3,
                        label=label_ci,
                        zorder=zorder)

    # Last but not least, plot the average ROC curve on top  of everything.
    plot_roc(roc=ret_mean, label=label, show_opt=show_opt,
             color=color, ax=ax, zorder=zorder + 3, **kwargs)
    plot_roc(roc=ret_mean_1, label=label_1, show_opt=show_opt,
             color=color_1, ax=ax, zorder=zorder + 3, **kwargs)
    plt.legend(title=legend)
    plt.savefig('./checkpoints/' + workname + '_auc.eps',format='eps', bbox_inches = 'tight', pad_inches = 0.1, dpi=1200)
    return ret_mean

In [ ]:
a = st.t.interval(alpha=0.95, df=len(roc), loc=np.mean(roc), scale=np.std(roc))

In [ ]:

# ctype, count= np.unique(trainval_target, return_counts=True)
# total_trainval_samples = len(trainval_target)
# binary_weight = np.asarray([ total_trainval_samples / k / len(ctype) for k in count])
# weight_dict = {}
# for i, c in enumerate(ctype):
#     weight_dict[c] = total_trainval_samples / count[i] / len(ctype)

In [ ]:
result_dict = {empty_list: [] for empty_list in range(5)}



In [ ]:
result_dict

{0: ['0.709', '(0.540-0.878)'], 1: [], 2: [], 3: [], 4: []}

In [ ]:
f.close()

In [ ]:
f = open("./MIMIC_RF.pkl",'rb')
object_file = pickle.load(f)

In [ ]:
object_file

{0: ['0.709', '(0.540-0.878)'], 1: [], 2: [], 3: [], 4: []}

In [ ]:
result_dict[0]

['0.709', '(0.540-0.878)']

In [ ]:
p_dict = xgb_bo.max['params']
p_dict['C']

0.12544361530068304

In [ ]:
# eval on itself and the other dataset 
regressor = LogisticRegression(penalty='elasticnet', dual=False,  C=0.1254, fit_intercept=True, intercept_scaling=1, \
                                class_weight= 'balanced', random_state=0, solver='saga', max_iter= 100, \
                                verbose=1, warm_start=False, n_jobs=None, l1_ratio=1.0)

regressor.fit(trainval_data, trainval_target)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


max_iter reached after 36 seconds


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   35.7s finished


LogisticRegression(C=0.1254, class_weight='balanced', l1_ratio=1.0,
                   penalty='elasticnet', random_state=0, solver='saga',
                   verbose=1)

In [ ]:
## bootstrapping on its own testset 
from tqdm import tqdm
from sklearn.metrics import average_precision_score
roc = []
prc = []
for i in tqdm(range(1000)):
    test_index = np.random.choice(len(test_target), 1000)
    test_i = [test_head[i] for i in test_index]
    test_t = test_target[test_index]
    test_data = np.stack(test_i).reshape((len(test_t), -1))
    test_pred_prob= regressor.predict_proba(test_data)
    test_roc = roc_auc_score(test_t, test_pred_prob[:, 1])
    test_prc = average_precision_score(test_t, test_pred_prob[:, 1])
    roc.append(test_roc)
    prc.append(test_prc)
#create 95% confidence interval for population mean weight


100%|██████████| 1000/1000 [00:20<00:00, 48.43it/s]


In [ ]:
import scipy.stats as st
#create 95% confidence interval for population mean weight
print('%.3f'%np.mean(roc))
print('(%.3f-%.3f)'%st.t.interval(alpha=0.95, df=len(roc), loc=np.mean(roc), scale=np.std(roc)))
print('%.3f'%np.mean(prc))
print('(%.3f-%.3f)'%st.t.interval(alpha=0.95, df=len(prc), loc=np.mean(prc), scale=np.std(prc)))

0.846
(0.810-0.882)
0.613
(0.532-0.695)


In [ ]:
# load the other 
# eicu test loader 
# load EICU DATA 
import importlib
from tqdm import tqdm
import scipy.stats as st
importlib.reload(prepare_data)
from sklearn.metrics import average_precision_score

data_label = np.load('/content/drive/My Drive/Colab Notebooks/MIMIC/Extract/MEEP/eICU_compile_0705_2022_1.npy', \
                allow_pickle=True).item()

etrain_head = data_label['train_head']
estatic_train_filter = data_label['static_train_filter']
# train_sofa_tail = data_label['train_sofa_tail']
# train_sofa_head = data_label['train_sofa_head']
edev_head = data_label['dev_head']
estatic_dev_filter = data_label['static_dev_filter']
# dev_sofa_tail = data_label['dev_sofa_tail']
# dev_sofa_head =['dev_sofa_head']
etest_head = data_label['test_head']
estatic_test_filter = data_label['static_test_filter']
# test_sofa_tail = data_label['test_sofa_tail']
# test_sofa_head =['test_sofa_head']
es_train = np.stack(estatic_train_filter, axis=0)
es_dev = np.stack(estatic_dev_filter, axis=0)
es_test = np.stack(estatic_test_filter, axis=0) 

In [ ]:
if args.target_index == 0 and args.filter_los == True:

    print('Before filtering, train size is %d'%(len(etrain_head)))
    es_train, etrain_data = filter_los(es_train, etrain_head, args.thresh, args.gap)
    es_dev, edev_data = filter_los(es_dev, edev_head, args.thresh, args.gap)
    es_test, etest_data = filter_los(es_test, etest_head, args.thresh, args.gap)
    print('After filtering, train size is %d'%(len(etrain_head)))
    etrain_label = es_train[:, 0]
    edev_label= es_dev[:, 0]
    etest_label = es_test[:, 0]

elif args.target_index == 1:
    etrain_data, etrain_label = filter_arf(args, etrain_head)
    edev_data, edev_label = filter_arf(args, edev_head)
    etest_data, etest_label = filter_arf(args, etest_head)

elif args.target_index == 2:
    etrain_data, etrain_label = filter_shock(args, etrain_head)
    edev_data, edev_label = filter_shock(args, edev_head)
    etest_data, etest_label = filter_shock(args, etest_head)

crossval_target = np.concatenate((etrain_label, edev_label, etest_label), axis= 0)
crossval_head = np.stack(etrain_data + edev_data + etest_data, axis=0)

In [ ]:
# bootstrapping
## bootstrapping on its own testset 
for i in tqdm(range(1000)):
    test_index = np.random.choice(len(crossval_target), 1000)
    test_i = [crossval_head[i] for i in test_index]
    test_t = crossval_target[test_index].astype(int)
    test_data = np.stack(test_i).reshape((len(test_t), -1))
    test_pred_prob= regressor.predict_proba(test_data)
    test_roc = roc_auc_score(test_t, test_pred_prob[:, 1])
    test_prc = average_precision_score(test_t, test_pred_prob[:, 1])
    roc.append(test_roc)
    prc.append(test_prc)
#create 95% confidence interval for population mean weight
print('%.3f'%np.mean(roc))
print('(%.3f-%.3f)'%st.t.interval(alpha=0.95, df=len(roc), loc=np.mean(roc), scale=np.std(roc)))
print('%.3f'%np.mean(prc))
print('(%.3f-%.3f)'%st.t.interval(alpha=0.95, df=len(prc), loc=np.mean(prc), scale=np.std(prc)))

100%|██████████| 1000/1000 [00:09<00:00, 103.80it/s]

0.709
(0.540-0.878)
0.264
(0.258-0.271)


In [ ]:
np.mean(prc)

0.44734120207157296

In [ ]:
import matplotlib.pyplot as plt
plt.hist(prc)

In [ ]:
trainval_target = np.concatenate((train_target, dev_target), axis=0)
trainval_data = np.stack(train_head + dev_head, axis=0).reshape((len(trainval_target), -1))

In [ ]:
from sklearn.metrics import roc_auc_score
auroc = roc_auc_score(test_target, test_pred[:, 1])

In [ ]:
import sklearn.metrics as metrics
import seaborn as sns
import matplotlib.pyplot as plt

num_class = 2
cm = metrics.confusion_matrix(test_target, test_pred)
label_x = None
label_y = None 
cf_matrix = cm/np.repeat(np.expand_dims(np.sum(cm, axis=1), axis=-1), num_class, axis=1)
group_counts = ['{0:0.0f}'.format(value) for value in cm.flatten()]
# percentage based on true label 
gr = (cm/np.repeat(np.expand_dims(np.sum(cm, axis=1), axis=-1), num_class, axis=1)).flatten()
group_percentages = ['{0:.2%}'.format(value) for value in gr]

labels = [f'{v1}\n{v2}' for v1, v2 in zip(group_percentages, group_counts)]

labels = np.asarray(labels).reshape(num_class, num_class)

if label_x is not None:
    xlabel = label_x
    ylabel = label_y
else:
    xlabel = ['Pred-%d'%i for i in range(num_class)]
    ylabel = ['%d'%i for i in range(num_class)]

sns.set(font_scale = 1.5)

hm = sns.heatmap(cf_matrix, annot=labels, fmt='', cmap = 'OrRd', \
annot_kws={"fontsize": 16}, xticklabels=xlabel, yticklabels=ylabel, cbar=False)
# hm.set(title=title)
fig = plt.gcf()
plt.show()